In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [2]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from math import sqrt
from PIL import Image, ImageEnhance, ImageFilter, ImageOps

In [3]:
def create_grid(collection):
    length = len(collection)
    root = sqrt(length)
    root = int(root)
    
    column = row = root
    
    width, height = np.vectorize(lambda x: x.size)(collection)
    width = np.amax(width)
    height = np.amax(height)

    offset = 10
    
    grid = Image.new(
        'L', 
        color='white',
        size=(
            (column * width) + offset, 
            (row * height) + (offset * 2)
        )
    )
    
    for index, image in enumerate(collection, 0):
        grid.paste(
            image, 
            box=(
                (index % column * width) + offset, 
                (index // column * height) + (offset * 2)
            )
        )
        
    return grid

In [4]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy
    
    return spectrogram.generate()

In [5]:
def create_image(spectrogram):
    # spectrogram[
    #     np.where(spectrogram > [100])
    # ] = [255]
        
    buffer = BytesIO()
    image = Image.fromarray(~spectrogram)
    
    # median = ImageFilter.MaxFilter(size=3)
    # image = image.filter(median)

    return image

In [6]:
dataset = Dataset('segment')
dataframe = dataset.load()

In [7]:
path = SETTINGS.joinpath('dereverberate.json')
dereverberate = Settings.from_file(path)

In [8]:
path = SETTINGS.joinpath('spectrogram.json')
settings = Settings.from_file(path)

In [9]:
minimum = 10
maximum = minimum + 10

subset = dataframe.iloc[minimum:maximum]
# subset = dataframe.sample(100)

In [10]:
segment = subset.segment.to_numpy()

In [11]:
%%capture

np.frompyfunc(
    lambda x: x.normalize(),
    1,
    0
)(segment)

In [12]:
%%capture

np.frompyfunc(
    lambda x: x.reduce(),
    1,
    0
)(segment)

In [13]:
%%capture

np.frompyfunc(
    lambda x: x.dereverberate(dereverberate),
    1,
    0
)(segment)

In [14]:
%%capture

spectrogram = np.frompyfunc(
    lambda x, y: create_spectrogram(x, y),
    2,
    1
)(segment, settings)

In [15]:
image = np.frompyfunc(
    lambda x: create_image(x),
    1,
    1
)(spectrogram)

In [16]:
grid = create_grid(image)
grid.show()